# Test multi-target Mapper

In [ ]:
from pixasonics.core import App, Mapper
from pixasonics.features import MeanChannelValue
from pixasonics.synths import Theremin, FilteredNoise, SimpleFM
from IPython.display import display, Audio

## Multi-target, single channel

In [ ]:
app0 = App()
app0.load_image_file("images/test.jpg")
mean_red = MeanChannelValue(filter_channels=0, name="MeanRed")
app0.attach(mean_red)
theremin0 = Theremin()
app0.attach(theremin0)
filtered_noise0 = FilteredNoise()
app0.attach(filtered_noise0)
red2freqs = Mapper(mean_red, [theremin0["frequency"], filtered_noise0["cutoff"]], name="Red2Freqs")
app0.attach(red2freqs)

In [ ]:
red2freqs.in_low, red2freqs.in_high, red2freqs.out_low, red2freqs.out_high, red2freqs.exponent

In [ ]:
red2freqs.out_low = [60, 60]
red2freqs.out_low

In [ ]:
red2freqs.out_high = [4000, 4000]
red2freqs.out_high

In [ ]:
red2freqs.exponent = [2, 2]
red2freqs.exponent = 2
red2freqs.exponent

In [ ]:
# test NRT
duration = 5
my_timeline = [
    (0, {
        "probe_width": 50,
        "probe_height": 50,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_x": 499,
        "probe_y": 499,
    })
]

target_filename = "multitarget_nrt_test.wav"

app0.render_timeline_to_file(my_timeline, target_filename)

display(Audio(target_filename))

## Multi-target, multi-channel

In [ ]:
app1 = App()
app1.load_image_file("images/test.jpg")
mean_pix = MeanChannelValue()
app1.attach(mean_pix)
num_channels = 3
theremin1 = Theremin([440 for _ in range(num_channels)])
app1.attach(theremin1)
filtered_noise1 = FilteredNoise(cutoff=[440 for _ in range(num_channels)])
app1.attach(filtered_noise1)
red2freqs1 = Mapper(mean_pix, [theremin1["frequency"], filtered_noise1["cutoff"]], name="Red2Freqs1")
app1.attach(red2freqs1)

In [ ]:
# test NRT
duration = 5
my_timeline = [
    (0, {
        "probe_width": 50,
        "probe_height": 50,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_x": 499,
        "probe_y": 499,
    })
]

target_filename = "multitarget_nrt_test.wav"

app1.render_timeline_to_file(my_timeline, target_filename)

display(Audio(target_filename))

# Test custom Feature + custom Mapper

In [ ]:
from pixasonics.core import App, Mapper
from pixasonics.features import MeanChannelValue, Feature
from pixasonics.synths import Theremin, FilteredNoise, SimpleFM
from IPython.display import display, Audio
import numpy as np
import os
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KDTree
from sklearn.decomposition import IncrementalPCA
import matplotlib.pyplot as plt

In [ ]:
app = App()

In [ ]:
app.load_image_file("images/cellular_dataset/merged_8bit/Timepoint_001_220518-ST_C03_s1.jpg")

In [ ]:
# combine red and green channels and all layers
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = os.listdir(img_folder)
imgs_red = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs_green = [f for f in img_files if f.endswith("w1.TIF")] # only green channel images
imgs = []
for img_red, img_green in zip(imgs_red, imgs_green):
    img_path_red = os.path.join(img_folder, img_red)
    img_path_green = os.path.join(img_folder, img_green)
    img_red = Image.open(img_path_red)
    img_green = Image.open(img_path_green)
    img_red = np.array(img_red)
    img_green = np.array(img_green)
    img = np.stack([img_red, img_green], axis=-1) # now the last dimension is the channel dimension
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app.load_image_data(img) # load as numpy array

In [ ]:
# synthmaps_pca_mel_json = "/Volumes/T7RITMO/synthmaps_code/data/pca_mels_mean.json"
synthmaps_pca_mel_json = "/Volumes/T7RITMO/synthmaps_code/data/pca_perceptual.json"
# synthmaps_pca_mel_json = "/Volumes/T7RITMO/synthmaps_code/data/pca_encodec.json"
# synthmaps_pca_mel_json = "/Volumes/T7RITMO/synthmaps_code/data/pca_clap.json"
with open(synthmaps_pca_mel_json, "r") as f:
    pca_mel_data = json.load(f)
print(pca_mel_data.keys())

In [ ]:
def fluid_dataset2array(
        dataset: dict,
) -> np.ndarray:
    """
    Convert a json dataset to a numpy array.

    Args:
        dataset (dict): The json dataset to convert.

    Returns:
        np.ndarray: The numpy array.
    """
    num_cols = dataset["cols"]
    num_rows = len(dataset["data"])
    out_array = np.zeros((num_rows, num_cols))
    for i in range(num_rows):
        out_array[i] = np.array(dataset["data"][str(i)])
    return out_array

In [ ]:
pca_mel_data_array = fluid_dataset2array(pca_mel_data)
print(pca_mel_data_array.shape)

In [ ]:
synthmaps_scaler = MinMaxScaler()
pca_mel_data_scaled = synthmaps_scaler.fit_transform(pca_mel_data_array)
print(pca_mel_data_scaled.shape)
print(pca_mel_data_scaled.min(), pca_mel_data_scaled.max())

In [ ]:
fm_params_json = "/Volumes/T7RITMO/synthmaps_code/data/fm_params.json"
with open(fm_params_json, "r") as f:
    fm_params_data = json.load(f)
print(fm_params_data.keys())
fm_params_data_array = fluid_dataset2array(fm_params_data)
print(fm_params_data_array.shape)

In [ ]:
class PCA2D(Feature):
    def __init__(self, name="PCA2D"):
        super().__init__(name=name)
        self.pca = None
        self.pca_scaler = None
        self._original_shape = None
        self._transformed_points = None

    def _reshape_for_pca(self, mat):
        """Helper to reshape 4D matrix (H, W, Ch, L) to 2D by concatenating the Channel and Layer dimensions"""
        mat_reshaped = mat.reshape(mat.shape[0], mat.shape[1], -1)
        return mat_reshaped.reshape(-1, mat_reshaped.shape[-1])

    def process_image(self, mat):
        self._original_shape = mat.shape
        features = self._reshape_for_pca(mat)
        self.pca = IncrementalPCA(n_components=2)
        self.pca.fit(features)
        self.pca_scaler = MinMaxScaler(feature_range=(0.1, 0.9))
        self._transformed_points = self.pca.transform(features)
        self.pca_scaler.fit(self._transformed_points)
        projected_scaled = self.pca_scaler.transform(self._transformed_points)
        return projected_scaled.T
    
    def compute(self, mat):
        if self.pca is None:
            raise ValueError("PCA model has not been fitted. Call process_image first.")
        features = self._reshape_for_pca(mat)
        projected = self.pca.transform(features)
        projected_scaled = self.pca_scaler.transform(projected)
        projected_scaled_mean = projected_scaled.mean(axis=0, keepdims=False)
        return projected_scaled_mean


pca_2d = PCA2D()
app.attach(pca_2d)

In [ ]:
#plot the pca space
plt.figure(figsize=(10, 5))

plt.subplot(121)

points = pca_2d._transformed_points
print(points.shape)
plt.scatter(points[:, 0], points[:, 1], alpha=0.5)
plt.title('PCA Space')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

In [ ]:
fm = SimpleFM()
app.attach(fm)

In [ ]:
class PCA2FMParams(Mapper):
    def __init__(self, source, target_fm, name="PCA2FMParams"):
        super().__init__(source, [target_fm["carrier_freq"], target_fm["harm_ratio"], target_fm["mod_index"]], name=name)
        self.kdtree = KDTree(pca_mel_data_scaled)

    def map(self, in_data):
        nearest_idx = self.kdtree.query(in_data.T, return_distance=False)[0][0]
        fm_params = fm_params_data_array[nearest_idx]
        return fm_params

pca2fm_params = PCA2FMParams(pca_2d, fm)
app.attach(pca2fm_params)

# Custom synth example

In [ ]:
from pixasonics.core import App, Mapper
from pixasonics.features import MeanChannelValue
from pixasonics.synths import Synth

In [ ]:
app = App()
app.load_image_file("images/test.jpg")
mean_pix = MeanChannelValue()
app.attach(mean_pix)


## Custom synth: AM

In [ ]:
import signalflow as sf

In [ ]:
class SimpleAMPatch(sf.Patch):
    def __init__(self, carr_freq=440, mod_freq=1, mod_depth=0.25):
        super().__init__()
        carr_freq = self.add_input("carrier_freq", carr_freq)
        mod_freq = self.add_input("mod_freq", mod_freq)
        mod_depth = self.add_input("mod_depth", mod_depth)
        modulator = (sf.SineOscillator(mod_freq) * mod_depth) + (1 - mod_depth)
        carrier = sf.SineOscillator(carr_freq)
        out = carrier * modulator
        self.set_output(out)

In [ ]:
spec = SimpleAMPatch().to_spec()
spec

### One-off version

In [ ]:
# create a dict for params
am_params = {
    "carrier_freq": {
        "min": 20,
        "max": 8000,
        "unit": "Hz",
        "scale": "log"
    },
    "mod_freq": {
        "min": 0.1,
        "max": 100,
        "unit": "Hz",
        "scale": "log"
    },
    "mod_depth": {
        "min": 0,
        "max": 1,
    }
}

In [ ]:
simple_am_synth = Synth(spec, am_params, name="SimpleAM", add_amplitude=True, add_panning=True)
app.attach(simple_am_synth)

### Long-term version

In [ ]:
class SimpleAM(Synth):
    def __init__(
            self,
            carrier_frequency=440,
            modulator_frequency=1,
            modulator_depth=0.25,
            name="SimpleAM"
    ):
        _spec = SimpleAMPatch(
                carr_freq=carrier_frequency,
                mod_freq=modulator_frequency,
                mod_depth=modulator_depth
        ).to_spec()
        _params = {
            "carrier_freq": {
                "min": 20,
                "max": 8000,
                "unit": "Hz",
                "scale": "log"
            },
            "mod_freq": {
                "min": 0.1,
                "max": 100,
                "unit": "Hz",
                "scale": "log"
            },
            "mod_depth": {
                "min": 0,
                "max": 1,
            }
        }
        # call the parent constructor
        super().__init__(_spec, params_dict=_params, name=name, add_amplitude=True, add_panning=True)

def __repr__(self):
    return f"SimpleAM {self.id}: {self.name}"

In [ ]:
simple_am = SimpleAM()
app.attach(simple_am)

In [ ]:
simple_am.ui

# Custom Mapper example

In [ ]:
class MyRGB2AMMapper(Mapper):
    def __init__(self, source, target_am, name="MyRGB2AMMapper"):
        super().__init__(source, [target_am["carrier_freq"], target_am["mod_freq"], target_am["mod_depth"]], name=name)
    
    def map(self, in_data):
        r, g, b = in_data
        # map red to carrier frequency
        carr_freq = np.interp(r, [0, 255], [20, 8000])
        # map green to modulator frequency
        mod_freq = np.interp(g, [0, 255], [0.1, 100])
        # map blue to modulator depth
        mod_depth = np.interp(b, [0, 2], [0, 1])
        return [carr_freq, mod_freq, mod_depth]

In [ ]:
my_rgb_2_am = MyRGB2AMMapper(mean_pix, simple_am)
app.attach(my_rgb_2_am)

# Test scaling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pixasonics.core import App, Mapper
from pixasonics.features import MeanChannelValue
from pixasonics.synths import Theremin
from pixasonics.utils import scale_array_exp
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
app_scaletest = App()
app_scaletest.load_image_file("images/test.jpg")
mean_red = MeanChannelValue(filter_channels=0, name="MeanRed")
app_scaletest.attach(mean_red)
theremin = Theremin()
app_scaletest.attach(theremin)
red2freq = Mapper(mean_red, [theremin["frequency"]], name="Red2Freqs")
app_scaletest.attach(red2freq)

In [ ]:
red2freq.in_low, red2freq.in_high, red2freq.out_low, red2freq.out_high, red2freq.exponent

In [ ]:
red2freq.exponent = 1

In [ ]:
def test_scale(in_val: float, exponent: float) -> float:
    return scale_array_exp(
        np.array([in_val], dtype=np.float64),
        np.array([0], dtype=np.float64),
        np.array([255], dtype=np.float64),
        np.array([60], dtype=np.float64),
        np.array([4000], dtype=np.float64),
        exponent
        )

In [ ]:
interact(test_scale, in_val=(0, 255, 0.01), exponent=(1, 2, 0.01))